# setting

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

!pip install datasets==2.14.0

In [ ]:
import pandas as pd
import os
import torch
import numpy as np
import random
from torch.utils.data import Dataset
from transformers import FNetTokenizer, FNetForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from datasets import Dataset
import time
import re
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [ ]:
# Seed 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 0
set_seed(seed)

In [ ]:
path = '/kaggle/input/'
sample_SEED = 0

# data

In [ ]:
data = pd.read_json(path+'data.json')

def split_data(data, sample_SEED, fixSEED=0):
  # normal bot split
  normal = data[data['restrict']==0].reset_index(drop=True)
  bot = data[data['restrict']==1].reset_index(drop=True)
  # normal underampling
  random.seed(sample_SEED)
  sample = random.sample(list(range(len(normal))), len(bot))
  normal = normal.loc[sample].reset_index(drop=True)
  data = pd.concat([normal, bot], axis=0)
  data['restrict'].value_counts()
  # train valid test split
  train, test = train_test_split(data, test_size=0.3, random_state=fixSEED, stratify = data['restrict'])
  train, valid = train_test_split(train, test_size=0.3, random_state=fixSEED, stratify = train['restrict'])
  train = train.reset_index(drop=True)
  valid = valid.reset_index(drop=True)
  test = test.reset_index(drop=True)
  # 1:1 fix
  random.seed(fixSEED)
  move = random.sample(list(valid[valid['restrict']==1].index), 1)
  test = pd.concat([test, valid.loc[move]], axis=0).reset_index(drop=True)
  valid = valid.drop(move, axis=0).reset_index(drop=True)
  return train, valid, test

# data split
train, valid, test = split_data(data, sample_SEED)

# model setting

In [ ]:
# model, tokenizer

tokenizer = FNetTokenizer.from_pretrained("google/fnet-base")
model = FNetForSequenceClassification.from_pretrained("google/fnet-base", num_labels=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



# training

In [ ]:
def extract_first_two_sentences(text):
    # Using regular expression to find sentences
    sentences = re.split(r'(?<=\.)\s+', text)
    # Return the first two sentences
    return ' '.join(sentences[:2])

# Apply the function to the text_column and create a new column
train['base'] = train['longlongtext'].apply(extract_first_two_sentences)
valid['base'] = valid['longlongtext'].apply(extract_first_two_sentences)
test['base'] = test['longlongtext'].apply(extract_first_two_sentences)




df_train = train.loc[:,['base', 'restrict']]
df_valid = valid.loc[:,['base', 'restrict']]
df_test = test.loc[:,['base', 'restrict']]

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
test_dataset = Dataset.from_pandas(df_test)


os.environ["WANDB_DISABLED"] = "true"


# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['base'], padding="max_length", max_length=512, truncation=True)


# TrainingArguments에 알맞게 데이터 처리
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("restrict", "labels")
valid_dataset = valid_dataset.rename_column("restrict", "labels")
test_dataset = test_dataset.rename_column("restrict", "labels")

train_dataset = train_dataset.remove_columns(['base'])
valid_dataset = valid_dataset.remove_columns(['base'])
test_dataset = test_dataset.remove_columns(['base'])

train_dataset.set_format(type='torch', columns=['input_ids', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'labels'])


# Trainer 설정
training_args = TrainingArguments(
    output_dir='./kaggle/working/results',         
    per_device_train_batch_size=8,   
    per_device_eval_batch_size=8,    
    num_train_epochs=4,             
    logging_dir='./kaggle/working/logs',           
    evaluation_strategy="epoch",     
    report_to = None,
    
)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    
    # Compute the confusion matrix
    conf_matrix = confusion_matrix(labels, preds)
    
    # Return metrics excluding confusion matrix
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }, conf_matrix

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,  # Pass the validation dataset
    compute_metrics=lambda p: compute_metrics(p)[0]
)

# 훈련 시간 측정 시작
start_time = time.time()

# 학습 시작 
trainer.train()

# 훈련 시간 측정 종료 
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total Training Time: {total_training_time:.2f} seconds")

# Final Prediction
test_results, test_conf_matrix = compute_metrics(trainer.predict(test_dataset))
print(f"Test results: {test_results}")
print("Confusion Matrix (Test):")
print(test_conf_matrix)
